<a href="https://colab.research.google.com/github/Miku0204/Michigan-Computer-Vision/blob/main/KNN%20Classifier/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-up code

In [1]:
user_email = "your_github_email"
user_name  = "your_github_name"

!git config --global user.email {user_email}
!git config --global user.name {user_name}

In [ ]:
repository = "Michigan-Computer-Vision"
git_token = "your github token"

!git clone https://{git_token}@github.com/{user_name}/{repository}